In [ ]:
# Check and install required packages if needed
import sys
import subprocess
import pkg_resources

required_packages = [
    'nltk',
    'numpy',
    'pandas',
    'matplotlib',
    'seaborn',
    'textblob',
    'wordcloud',
    'scikit-learn',
    'gensim',
    'SpeechRecognition'
]

def install_packages(packages):
    """Install packages using pip"""
    for package in packages:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully!")

# Check for missing packages
installed_packages = {pkg.key for pkg in pkg_resources.working_set}
missing_packages = [pkg for pkg in required_packages if pkg.lower() not in installed_packages]

# Install missing packages
if missing_packages:
    print(f"The following packages are missing and will be installed: {', '.join(missing_packages)}")
    try:
        install_packages(missing_packages)
        print("All required packages are now installed.")
    except Exception as e:
        print(f"Error installing packages: {e}")
        print("Please install the missing packages manually before running this notebook.")
else:
    print("All required packages are already installed.")


In [ ]:
# Import necessary libraries

# Core libraries
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data with error handling
def download_nltk_data():
    resources = ['punkt', 'stopwords', 'wordnet', 'vader_lexicon']
    for resource in resources:
        try:
            print(f"Downloading NLTK resource: {resource}...")
            nltk.download(resource, quiet=True)
            print(f"Successfully downloaded {resource}")
        except Exception as e:
            print(f"Error downloading {resource}: {e}")
            print(f"You might need to manually download this resource using nltk.download('{resource}')")

# Call the function to download NLTK data
download_nltk_data()

# NLTK specific imports
try:
    from nltk.tokenize import word_tokenize, sent_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer, WordNetLemmatizer
    from nltk.probability import FreqDist
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
except ImportError as e:
    print(f"Error importing NLTK modules: {e}")
    print("Make sure all NLTK resources are properly downloaded.")

# Import TextBlob
from textblob import TextBlob

# Visualization
from wordcloud import WordCloud

# Machine Learning
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Topic Modeling
from gensim import corpora
from gensim.models import LdaModel

# Speech Recognition
import speech_recognition as sr

# Set style for plots
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Speech and Sentiment Analysis Project

This notebook demonstrates various Natural Language Processing (NLP) techniques using Python libraries. We'll cover:

1. **Speech-to-Text Conversion**: Convert spoken language to text using SpeechRecognition
2. **Text Preprocessing**: Clean and normalize text data
3. **Exploratory Text Analysis**: Analyze text statistics and visualize patterns
4. **Sentiment Analysis**: Determine sentiment using multiple approaches
5. **Topic Modeling**: Extract topics from text collections
6. **Text Classification**: Build machine learning models to classify text

Let's dive in!

## 1. Speech-to-Text Conversion

First, let's set up the speech recognition functionality. This allows us to convert spoken language to text for further analysis.

In [ ]:
def recognize_speech():
    """Record speech and convert to text using Google's Speech Recognition API"""
    
    # Check if speech recognition is available
    try:
        # Initialize recognizer
        recognizer = sr.Recognizer()
        
        # Check if microphone is available
        try:
            mics = sr.Microphone.list_microphone_names()
            if not mics:
                print("No microphones found. Make sure your microphone is connected.")
                return ""
            print(f"Available microphones: {len(mics)}")
        except Exception as e:
            print(f"Error checking microphones: {e}")
        
        print("Please speak something...")
        
        # Try to capture speech from microphone
        try:
            with sr.Microphone() as source:
                print("Adjusting for ambient noise...")
                recognizer.adjust_for_ambient_noise(source, duration=1)
                print("Listening...")
                audio = recognizer.listen(source, timeout=5)
                print("Processing...")
                
            # Use Google Speech Recognition to convert audio to text
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
            return text
        
        except sr.WaitTimeoutError:
            print("No speech detected within the timeout period.")
            return ""
        except sr.UnknownValueError:
            print("Could not understand audio")
            return ""
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return ""
        except Exception as e:
            print(f"Error during speech recognition: {e}")
            return ""
    except Exception as e:
        print(f"Error setting up speech recognition: {e}")
        print("Make sure SpeechRecognition and PyAudio are installed correctly.")
        return ""

# Note: Uncomment to use the speech recognition
# speech_text = recognize_speech()

# For demonstration purposes, we'll use sample texts
sample_texts = [
    "I absolutely loved the movie. The acting was superb and the plot was engaging throughout.",
    "The customer service was terrible. I waited for hours and still did not get help.",
    "The product is okay, but it could be better. Some features work well while others need improvement.",
    "I'm really excited about the upcoming conference. The speakers lineup looks amazing!",
    "The food at the restaurant was mediocre, but the service was excellent."
]

# Create a DataFrame with sample texts for demonstration
df = pd.DataFrame(sample_texts, columns=['text'])
df.head()

## 2. Text Preprocessing

Before analyzing text data, we need to clean and normalize it. This typically involves:

- Converting to lowercase
- Removing punctuation and special characters
- Tokenization (splitting text into words)
- Removing stopwords (common words like 'and', 'the', etc.)
- Stemming or lemmatization (reducing words to their root form)

In [ ]:
# Initialize stopwords, stemmer and lemmatizer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    """Preprocess text by applying various cleaning operations"""
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return {
        'original_text': text,
        'tokens': tokens,
        'lemmatized_tokens': lemmatized_tokens,
        'processed_text': ' '.join(lemmatized_tokens)
    }

# Apply preprocessing to our sample texts
df['preprocessed'] = df['text'].apply(lambda x: preprocess_text(x))

# Extract processed text for further analysis
df['processed_text'] = df['preprocessed'].apply(lambda x: x['processed_text'])

# Display results
for i, row in df.iterrows():
    print(f"Original: {row['text']}")
    print(f"Processed: {row['processed_text']}")
    print("-" * 80)

## 3. Exploratory Text Analysis

Now let's explore the text data by analyzing word frequencies, text statistics, and creating visualizations.

In [ ]:
def get_text_stats(text_series):
    """Calculate various statistics for a series of texts"""
    
    # Text length statistics
    text_lengths = text_series.str.len()
    
    # Word count statistics
    word_counts = text_series.apply(lambda x: len(word_tokenize(x)))
    
    # Sentence count statistics
    sentence_counts = text_series.apply(lambda x: len(sent_tokenize(x)))
    
    # Average word length
    avg_word_lengths = text_series.apply(lambda x: np.mean([len(word) for word in word_tokenize(x)]) if word_tokenize(x) else 0)
    
    # Create a DataFrame with statistics
    stats_df = pd.DataFrame({
        'text_length': text_lengths,
        'word_count': word_counts,
        'sentence_count': sentence_counts,
        'avg_word_length': avg_word_lengths
    })
    
    return stats_df

# Get text statistics
text_stats = get_text_stats(df['text'])

# Display summary statistics
print("Text Statistics Summary:")
print(text_stats.describe())

# Create visualizations for text statistics
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
sns.histplot(text_stats['text_length'], kde=True)
plt.title('Text Length Distribution')

plt.subplot(2, 2, 2)
sns.histplot(text_stats['word_count'], kde=True)
plt.title('Word Count Distribution')

plt.subplot(2, 2, 3)
sns.histplot(text_stats['sentence_count'], kde=True)
plt.title('Sentence Count Distribution')

plt.subplot(2, 2, 4)
sns.histplot(text_stats['avg_word_length'], kde=True)
plt.title('Average Word Length Distribution')

plt.tight_layout()
plt.show()

# Word frequency analysis
all_words = ' '.join(df['processed_text']).split()
word_freq = Counter(all_words)

# Get top N most common words
top_n = 20
most_common_words = word_freq.most_common(top_n)

# Create DataFrame for plotting
common_words_df = pd.DataFrame(most_common_words, columns=['word', 'count'])

# Plot word frequencies
plt.figure(figsize=(12, 6))
sns.barplot(x='word', y='count', data=common_words_df)
plt.title(f'Top {top_n} Most Common Words')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Create word cloud
wc = WordCloud(width=800, height=400, background_color='white', max_words=100).generate(' '.join(df['processed_text']))

plt.figure(figsize=(12, 8))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Processed Text')
plt.show()

## 4. Sentiment Analysis

Sentiment analysis determines whether a piece of text expresses positive, negative, or neutral sentiment. We'll use multiple approaches for comparison.

In [ ]:
def analyze_sentiment_vader(text):
    """Analyze sentiment using NLTK's VADER SentimentIntensityAnalyzer"""
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(text)
    
    # Determine overall sentiment based on compound score
    if sentiment_scores['compound'] >= 0.05:
        sentiment = 'Positive'
    elif sentiment_scores['compound'] <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'
    
    return {
        'neg': sentiment_scores['neg'],
        'neu': sentiment_scores['neu'],
        'pos': sentiment_scores['pos'],
        'compound': sentiment_scores['compound'],
        'sentiment': sentiment
    }

def analyze_sentiment_textblob(text):
    """Analyze sentiment using TextBlob"""
    analysis = TextBlob(text)
    
    # TextBlob's polarity ranges from -1 (negative) to 1 (positive)
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity
    
    # Determine sentiment based on polarity
    if polarity > 0.05:
        sentiment = 'Positive'
    elif polarity < -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'
    
    return {
        'polarity': polarity,
        'subjectivity': subjectivity,
        'sentiment': sentiment
    }

# Apply sentiment analysis to our texts
df['vader_sentiment'] = df['text'].apply(analyze_sentiment_vader)
df['textblob_sentiment'] = df['text'].apply(analyze_sentiment_textblob)

# Extract sentiment labels for comparison
df['vader_label'] = df['vader_sentiment'].apply(lambda x: x['sentiment'])
df['textblob_label'] = df['textblob_sentiment'].apply(lambda x: x['sentiment'])

# Display results
print("Sentiment Analysis Results:")
for i, row in df.iterrows():
    print(f"Text: {row['text']}")
    print(f"VADER: {row['vader_label']} (compound: {row['vader_sentiment']['compound']:.3f})")
    print(f"TextBlob: {row['textblob_label']} (polarity: {row['textblob_sentiment']['polarity']:.3f}, subjectivity: {row['textblob_sentiment']['subjectivity']:.3f})")
    print("-" * 80)

# Visualize sentiment distribution
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.countplot(x='vader_label', data=df)
plt.title('Sentiment Distribution (VADER)')

plt.subplot(1, 2, 2)
sns.countplot(x='textblob_label', data=df)
plt.title('Sentiment Distribution (TextBlob)')

plt.tight_layout()
plt.show()

# Compare VADER compound scores with TextBlob polarity
plt.figure(figsize=(10, 6))
plt.scatter(
    [x['compound'] for x in df['vader_sentiment']],
    [x['polarity'] for x in df['textblob_sentiment']],
    alpha=0.7
)
plt.grid(True)
plt.xlabel('VADER Compound Score')
plt.ylabel('TextBlob Polarity')
plt.title('VADER vs. TextBlob Sentiment Scores')
plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='r', linestyle='-', alpha=0.3)
plt.show()

## 5. Topic Modeling

Topic modeling is a technique to discover abstract topics in a collection of documents. We'll use Latent Dirichlet Allocation (LDA) for this task.

In [ ]:
def perform_lda_topic_modeling(texts, num_topics=3, num_words=5):
    """Perform LDA topic modeling on a list of preprocessed texts"""
    # Convert texts to list of lists of words
    texts_as_lists = [text.split() for text in texts]
    
    # Create dictionary
    dictionary = corpora.Dictionary(texts_as_lists)
    
    # Create document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts_as_lists]
    
    # Build LDA model
    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        random_state=42,
        passes=10
    )
    
    # Extract topics
    topics = lda_model.print_topics(num_words=num_words)
    
    # Get document-topic distribution
    doc_topics = [lda_model[doc] for doc in corpus]
    
    return {
        'model': lda_model,
        'topics': topics,
        'doc_topics': doc_topics,
        'corpus': corpus,
        'dictionary': dictionary
    }

# For robust topic modeling, we would need more documents
# For demonstration purposes, let's create some additional texts
additional_texts = [
    "The smartphone has an excellent camera and battery life.",
    "This laptop performance is outstanding for gaming and productivity.",
    "The hotel room was spacious with a beautiful view of the ocean.",
    "The conference speakers provided valuable insights on artificial intelligence.",
    "This restaurant serves delicious Italian cuisine with excellent wine pairings."
]

# Safely preprocess additional texts
preprocessed_additional_texts = []
for text in additional_texts:
    try:
        preprocessed_additional_texts.append(preprocess_text(text)['processed_text'])
    except Exception as e:
        print(f"Error preprocessing text: {e}")
        preprocessed_additional_texts.append("")

# Combine with our original texts
all_texts = list(df['processed_text']) + preprocessed_additional_texts

# Remove any empty strings that might have resulted from preprocessing errors
all_texts = [text for text in all_texts if text]

# Check if we have enough texts for topic modeling
if len(all_texts) < 3:
    print("Not enough texts for meaningful topic modeling. Need at least 3 documents.")
    num_topics = min(3, len(all_texts))
else:
    num_topics = 3

# Perform topic modeling
lda_results = perform_lda_topic_modeling(all_texts, num_topics=num_topics)

# Display topics
print(f"Top {num_topics} Topics:")
for i, topic in enumerate(lda_results['topics']):
    print(f"Topic #{i+1}: {topic}")
    
# Display document-topic distribution
print("\nDocument-Topic Distribution:")
for i, doc_topic in enumerate(lda_results['doc_topics']):
    # Sort topic distribution by weight
    sorted_topics = sorted(doc_topic, key=lambda x: x[1], reverse=True)
    
    # Get most dominant topic
    dominant_topic = sorted_topics[0] if sorted_topics else (0, 0)
    
    # Get original text (cutting it if too long)
    text = all_texts[i]
    if len(text) > 50:
        text = text[:50] + '...'
        
    print(f"Document #{i+1}: Dominant Topic: #{dominant_topic[0]+1} ({dominant_topic[1]:.2f})")
    print(f"   Text: {text}")

## 6. Text Classification

Now we'll use machine learning to build a text classifier based on sentiment. Since we've already done sentiment analysis, we'll use those labels as our target variable.

In [ ]:
# Since we have a small dataset, let's create more synthetic examples for demonstration purposes
def generate_synthetic_examples():
    positive_examples = [
        "I really enjoyed this product, it exceeded my expectations.",
        "The service was outstanding, I'll definitely come back.",
        "This is by far the best experience I've had with any company.",
        "The staff was friendly and very helpful throughout the process.",
        "Excellent quality and fast delivery, highly recommended!",
        "I'm very satisfied with my purchase, great value for money.",
        "The app is intuitive and has all the features I needed.",
        "Their customer support resolved my issue promptly and professionally."
    ]
    
    neutral_examples = [
        "The product works as expected, nothing special though.",
        "It's an average service, neither good nor bad.",
        "The quality is acceptable for the price point.",
        "I received exactly what was advertised, no surprises.",
        "The performance is standard, meets basic requirements.",
        "It's okay for occasional use but not for heavy usage.",
        "The interface is functional but could use some improvements.",
        "Not bad, but I've seen better options in this price range."
    ]
    
    negative_examples = [
        "This product is a complete waste of money.",
        "The customer service was rude and unhelpful.",
        "I'm very disappointed with the quality, it broke after a week.",
        "Definitely not worth the price, I regret this purchase.",
        "The app is buggy and crashes constantly, terrible user experience.",
        "I waited for hours and still didn't get proper assistance.",
        "Would not recommend to anyone, stay away from this service.",
        "The worst experience I've ever had with an online store."
    ]
    
    all_examples = []
    all_labels = []
    
    for examples, sentiment in zip([positive_examples, neutral_examples, negative_examples], 
                                 ['Positive', 'Neutral', 'Negative']):
        all_examples.extend(examples)
        all_labels.extend([sentiment] * len(examples))
    
    # Create DataFrame
    synthetic_df = pd.DataFrame({
        'text': all_examples,
        'sentiment': all_labels
    })
    
    # Apply preprocessing
    synthetic_df['processed_text'] = synthetic_df['text'].apply(lambda x: preprocess_text(x)['processed_text'])
    
    return synthetic_df

# Generate synthetic dataset
synthetic_df = generate_synthetic_examples()

# Display sample of the synthetic dataset
print(f"Synthetic dataset shape: {synthetic_df.shape}")
print(synthetic_df.head())

# Feature extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(synthetic_df['processed_text'])
y = synthetic_df['sentiment']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Train a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Make predictions
y_pred_nb = nb_classifier.predict(X_test)

# Evaluate the model
print("Multinomial Naive Bayes Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_nb):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_nb))

# Train a Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000, random_state=42)
lr_classifier.fit(X_train, y_train)

# Make predictions
y_pred_lr = lr_classifier.predict(X_test)

# Evaluate the model
print("\nLogistic Regression Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

# Plot confusion matrices
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

sns.heatmap(confusion_matrix(y_test, y_pred_nb), annot=True, fmt='d', ax=ax1, 
            cmap='Blues', xticklabels=nb_classifier.classes_, yticklabels=nb_classifier.classes_)
ax1.set_title('Confusion Matrix - Naive Bayes')
ax1.set_xlabel('Predicted')
ax1.set_ylabel('Actual')

sns.heatmap(confusion_matrix(y_test, y_pred_lr), annot=True, fmt='d', ax=ax2, 
            cmap='Blues', xticklabels=lr_classifier.classes_, yticklabels=lr_classifier.classes_)
ax2.set_title('Confusion Matrix - Logistic Regression')
ax2.set_xlabel('Predicted')
ax2.set_ylabel('Actual')

plt.tight_layout()
plt.show()

In [ ]:
def analyze_new_text(text, tfidf_vectorizer=None, classifier=None):
    """Analyze a new text input using our NLP pipeline"""
    
    print("\n==== Text Analysis ====\n")
    print(f"Original text: {text}")
    
    # Preprocess text
    preprocessed = preprocess_text(text)
    processed_text = preprocessed['processed_text']
    print(f"\nProcessed text: {processed_text}")
    
    # Get text statistics
    num_words = len(word_tokenize(text))
    num_chars = len(text)
    num_sentences = len(sent_tokenize(text))
    
    print(f"\nText Statistics:")
    print(f"- Characters: {num_chars}")
    print(f"- Words: {num_words}")
    print(f"- Sentences: {num_sentences}")
    print(f"- Average word length: {np.mean([len(word) for word in word_tokenize(text)]):.2f}")
    
    # Sentiment analysis
    vader_sentiment = analyze_sentiment_vader(text)
    textblob_sentiment = analyze_sentiment_textblob(text)
    
    print(f"\nSentiment Analysis:")
    print(f"- VADER: {vader_sentiment['sentiment']} (compound: {vader_sentiment['compound']:.3f})")
    print(f"- TextBlob: {textblob_sentiment['sentiment']} (polarity: {textblob_sentiment['polarity']:.3f}, "
          f"subjectivity: {textblob_sentiment['subjectivity']:.3f})")
    
    # Create a basic result dictionary
    result = {
        'original_text': text,
        'processed_text': processed_text,
        'statistics': {
            'chars': num_chars,
            'words': num_words,
            'sentences': num_sentences
        },
        'vader_sentiment': vader_sentiment,
        'textblob_sentiment': textblob_sentiment
    }
    
    # Only perform ML classification if vectorizer and classifier are provided
    if tfidf_vectorizer is not None and classifier is not None:
        try:
            # Transform text using the vectorizer
            text_vectorized = tfidf_vectorizer.transform([processed_text])
            predicted_sentiment = classifier.predict(text_vectorized)[0]
            probabilities = classifier.predict_proba(text_vectorized)[0]
            
            print(f"\nML Classification:")
            print(f"- Predicted sentiment: {predicted_sentiment}")
            print(f"- Prediction probabilities:")
            for i, label in enumerate(classifier.classes_):
                print(f"  - {label}: {probabilities[i]:.4f}")
                
            # Add ML classification to result
            result['ml_classification'] = {
                'prediction': predicted_sentiment,
                'probabilities': {label: prob for label, prob in zip(classifier.classes_, probabilities)}
            }
        except Exception as e:
            print(f"\nError during ML classification: {e}")
    else:
        print("\nML Classification: Skipped (vectorizer or classifier not provided)")
    
    return result

# Example usage with a new text
new_text = "I just received my new headphones and they sound absolutely amazing. The noise cancellation is effective and the battery lasts all day. Highly recommended!"

try:
    # Try to use the previously defined vectorizer and classifier if they exist
    analysis_result = analyze_new_text(new_text, tfidf_vectorizer, lr_classifier)
except NameError as e:
    # If they don't exist, just run without ML classification
    print(f"Note: {e}. Running without ML classification.")
    analysis_result = analyze_new_text(new_text)

## 7. Conclusion

In this project, we've explored various NLP techniques using Python libraries:

1. **Speech-to-Text Conversion**: We set up functionality to convert spoken language to text using SpeechRecognition.

2. **Text Preprocessing**: We implemented a comprehensive text preprocessing pipeline including tokenization, stopword removal, and lemmatization.

3. **Exploratory Text Analysis**: We analyzed text statistics and created visualizations to understand our text data better.

4. **Sentiment Analysis**: We compared different sentiment analysis approaches (VADER and TextBlob).

5. **Topic Modeling**: We used LDA to discover abstract topics in a collection of documents.

6. **Text Classification**: We built machine learning models to classify text based on sentiment.

This project demonstrates how NLP can be used to extract insights from text data. You can extend this project by:

- Incorporating more advanced NLP techniques like named entity recognition
- Using more sophisticated models like transformers (BERT, GPT, etc.) for sentiment analysis
- Creating a real-time speech processing application
- Building a larger dataset for more robust model training